In [1]:
import sys
print ('My Python Version =' + sys.version)

My Python Version =3.6.1 (default, Apr  4 2017, 09:40:21) 
[GCC 4.2.1 Compatible Apple LLVM 8.1.0 (clang-802.0.38)]


In [11]:
import requests
import json
PORT_NUMBER = 1234
IP = 'localhost'
BASE = 'http://' + IP + ':' + str(PORT_NUMBER) + '/v1/'
HEADERS = {'Content-Type': 'application/json'}

In [6]:
res = requests.get(BASE)

In [7]:
status_object = res.json();
print(json.dumps(status_object, indent=4))

{
    "apiVersion": "v1",
    "numberOfCores": 4,
    "memoryStatus": {
        "usedMemory": 1416,
        "freeMemory": 814,
        "totalMemory": 2230,
        "maxMemory": 3460
    }
}


In [8]:
print(status_object['apiVersion'])
print(status_object['memoryStatus']['usedMemory'])

v1
1416


In [10]:
# Start from a clean slate: remove all networks from current session
requests.delete(BASE + 'networks')

# Manually generates JSON as dictionary

empty_network = {
        'data': {
            'name': 'I\'m empty!'
        },
        'elements': {
            'nodes':[],
            'edges':[]
        }
}

res = requests.post(BASE + 'networks?collection=My%20Collection', data=json.dumps(empty_network), headers=HEADERS) 
net_suid = res.json()['networkSUID']
print('Empty network has SUID ' + str(net_suid))

Empty network has SUID 290710


In [12]:
import copy

# Create a copy of the empty network object
small_network = copy.deepcopy(empty_network)

# Sequence of letters (a-z)
seq_letters = list(map(chr, range(ord('a'), ord('z')+1)))

# Build nodes and edges (in functional way)
build_node = lambda x: {'data': { 'id': x }}
abc_nodes = list(map(build_node, seq_letters))

build_edge = lambda x: {'data': { 'source': x, 'target': 'a' }}
rand_edges = list(map(build_edge, seq_letters))

small_network['elements']['nodes'] = abc_nodes
small_network['elements']['edges'] = rand_edges
small_network['data']['name'] = 'A is the hub.'

# Uncomment this if you want to see the actual JSON object
#print(json.dumps(small_network, indent=4))

res = requests.post(BASE + 'networks?collection=My%20Collection', data=json.dumps(small_network), headers=HEADERS)
res_dict = res.json()
new_suid = res_dict['networkSUID']

# Apply layout
requests.get(BASE + 'apply/layouts/force-directed/' + str(new_suid))

<Response [200]>